### The main notebook for modelling the system

Writing a code to integrate the following system. This models the generation of vegetative stripes and the respective diffusion uphill. The variable $u$ denotes the plant biomass, a function of time and space. The variable $w$ denotes the level of surface water. The constant $a$ is the level of rainfall, $w$ the evaporation. 

This is the Klausmeier model: 

$$\frac{\partial u}{\partial t} = wu^2 - bu+ \frac{\partial^2 u }{\partial x^2}$$

$$
\frac{\partial w}{\partial t} = a - w - wu^2 + \nu \frac{\partial w}{\partial x}
$$

Note that by analysis we must have the $a \geq 2b$. The level of rainfall is greater of equal to the rate of plant death multiplied by 2. 

Want to write a function that takes initial conditions, domain size and total time and outputs the vegetation data over time and space. 

Start off with solving:

$$
\frac{\partial u}{\partial t} = \frac{\delta^2 u}{\delta x^2}
$$
$$
\frac{\partial w}{\partial t} = \nu \frac{\partial w}{\partial x}
$$

In [54]:
import numpy as np 
import scipy 
from  scipy.integrate import solve_ivp
#SOlve IVP for scipy, scipy.integrate? 

In [58]:
n = 51
x = np.linspace(0,1,n)
#Make u a gaussian
u = np.exp(-100*(x-0.5)**2)
print(u)
print(len(u))


nu = 4
dx = 1

DU = np.zeros((n,n))
DW = np.zeros((n,n))

np.fill_diagonal(DU,-2*1**-2)
np.fill_diagonal(DU[1:], 1**-2)
np.fill_diagonal(DU[:, 1:], 1**-2)
DU[0,-1] = 1**-2
DU[-1,0] = 1**-2
DU = DU*(dx**-2)
np.fill_diagonal(DW,-1)
np.fill_diagonal(DW[:,1:],1)
DW[0,-1] = 1
DW[-1,0] = 1
DW = DW*nu*(dx**-1)



[1.38879439e-11 9.85950558e-11 6.46143177e-10 3.90893843e-09
 2.18295780e-08 1.12535175e-07 5.35534780e-07 2.35257520e-06
 9.54016287e-06 3.57128496e-05 1.23409804e-04 3.93669041e-04
 1.15922917e-03 3.15111160e-03 7.90705405e-03 1.83156389e-02
 3.91638951e-02 7.73047404e-02 1.40858421e-01 2.36927759e-01
 3.67879441e-01 5.27292424e-01 6.97676326e-01 8.52143789e-01
 9.60789439e-01 1.00000000e+00 9.60789439e-01 8.52143789e-01
 6.97676326e-01 5.27292424e-01 3.67879441e-01 2.36927759e-01
 1.40858421e-01 7.73047404e-02 3.91638951e-02 1.83156389e-02
 7.90705405e-03 3.15111160e-03 1.15922917e-03 3.93669041e-04
 1.23409804e-04 3.57128496e-05 9.54016287e-06 2.35257520e-06
 5.35534780e-07 1.12535175e-07 2.18295780e-08 3.90893843e-09
 6.46143177e-10 9.85950558e-11 1.38879439e-11]
51


In [62]:
def spatial_differential(t, u):
    # Here, you perform the matrix multiplication (u @ DU) * v
    
    du_dt = DU@np.transpose(u) # Matrix multiplication and product with v

    return np.transpose(du_dt).flatten()

In [63]:
t_span = np.linspace(0,10,100)  # Example time span from 0 to 10

# Flatten u and v to be passed as a 1D array to solve_ivp
u_initial = u.flatten()
print(u_initial)

print(spatial_differential(0, u_initial))


# Integrate the spatial differential over time using solve_ivp
sol = solve_ivp(spatial_differential, t_span, u_initial)

len(sol.y)

[1.38879439e-11 9.85950558e-11 6.46143177e-10 3.90893843e-09
 2.18295780e-08 1.12535175e-07 5.35534780e-07 2.35257520e-06
 9.54016287e-06 3.57128496e-05 1.23409804e-04 3.93669041e-04
 1.15922917e-03 3.15111160e-03 7.90705405e-03 1.83156389e-02
 3.91638951e-02 7.73047404e-02 1.40858421e-01 2.36927759e-01
 3.67879441e-01 5.27292424e-01 6.97676326e-01 8.52143789e-01
 9.60789439e-01 1.00000000e+00 9.60789439e-01 8.52143789e-01
 6.97676326e-01 5.27292424e-01 3.67879441e-01 2.36927759e-01
 1.40858421e-01 7.73047404e-02 3.91638951e-02 1.83156389e-02
 7.90705405e-03 3.15111160e-03 1.15922917e-03 3.93669041e-04
 1.23409804e-04 3.57128496e-05 9.54016287e-06 2.35257520e-06
 5.35534780e-07 1.12535175e-07 2.18295780e-08 3.90893843e-09
 6.46143177e-10 9.85950558e-11 1.38879439e-11]
[ 8.47071119e-11  4.62841010e-10  2.71524714e-09  1.46578443e-08
  7.27849573e-08  3.32294009e-07  1.39404081e-06  5.37054725e-06
  1.89850991e-05  6.15242677e-05  1.82562282e-04  4.95300897e-04
  1.22632229e-03  2.764060

ValueError: too many values to unpack (expected 2)

In [48]:
#Plot this
import matplotlib.pyplot as plt
#Plot for t=1, over space 

#Plt this over time
sol.t

array([ 0.        ,  0.12898708,  1.41885784,  3.84637675,  6.84144157,
        7.73450238,  8.62756318,  9.56328552, 10.        ])

In [4]:
def ode_func(t, y):
    # Here, y is an array representing the state variables
    # For example, if y = [x, y], the function returns [dx/dt, dy/dt]
    dydt = np.zeros_like(y)
    dydt[0] = 2 * y[0]  # Example ODE: dx/dt = 2x
    dydt[1] = -0.5 * y[1]  # Example ODE: dy/dt = -0.5y
    return dydt

# Initial conditions
y0 = [1.0, 2.0]  # Initial values of x and y

# Time span
t_span = (0, 10)  # Solve from t=0 to t=10

# Solve the ODE
sol = solve_ivp(ode_func, t_span, y0, t_eval=np.linspace(0, 10, 100))



In [59]:
import numpy as np
from scipy.integrate import solve_ivp

n = 10
u = np.random.rand(n, 1)
w = np.random.rand(n, 1)

nu = 4
dx = 1

DU = np.zeros((n, n))
np.fill_diagonal(DU, -2 * 1 ** -2)
np.fill_diagonal(DU[1:], 1 ** -2)
np.fill_diagonal(DU[:, 1:], 1 ** -2)
DU[0, -1] = 1 ** -2
DU[-1, 0] = 1 ** -2
DU = DU * (dx ** -2)

DW = np.zeros((n, n))
np.fill_diagonal(DW, -1)
np.fill_diagonal(DW[:, 1:], 1)
DW[0, -1] = 1
DW[-1, 0] = 1
DW = DW * nu * (dx ** -1)

def deriv_variables(t, variables, DU, DW):
    u, w = variables
    deriv_u = DU @ u
    deriv_w = DW @ w
    return [deriv_u, deriv_w]

# Time span
t_span = (0, 10)

# Solve the system of ODEs
sol = solve_ivp(deriv_variables, t_span, [u, w], args=(DU, DW), t_eval=np.linspace(0, 10, 100))

# Now you can access the solution as sol.y[0] for u and sol.y[1] for w


ValueError: `y0` must be 1-dimensional.

In [ ]:

n = 10

DU = np.zeros((n,n))


#DU[1:-1, 1:-1] = -2 * 1**-2

# Off-diagonal elements
np.fill_diagonal(DU,-2*1**-2)
np.fill_diagonal(DU[1:], 1**-2)
np.fill_diagonal(DU[:, 1:], 1**-2)

# Boundary conditions
'''
DU[0, 0] = -2 * 1**-2
DU[0, 1] = 1**-2
#DU[1, 0] = 1**-2
DU[-1, -2] = 1**-2
DU[-1, -1] = -2 * 1**-2
'''
DU[0,-1] = 1**-2
DU[-1,0] = 1**-2

print(DU)
DU.shape


In [ ]:
from model_functions import *
import matplotlib.pyplot as plt


param = [2.2,0.45,182.5]
model = model(0.01,0.1,100,100,param)


u = np.ones(model.nx)
w = np.ones(model.nx)

#Set initial condition of u as some gaussian? with added noise


u_next,w_next = model.u_w_next(u,w)

#Plot u on domain 

plt.plot(u_next)
